# Deep Agents Workshop - Steps 10-12

This notebook demonstrates the transition from LangChain v1 core agents to **Deep Agents**.

## What are Deep Agents?

Deep Agents are an "agent harness" built on top of LangGraph that come with built-in capabilities for complex, multi-step tasks:

- 📋 **Planning**: `write_todos` tool for task decomposition
- 📁 **File System**: `ls`, `read_file`, `write_file`, `edit_file` for context management
- 🤖 **Subagents**: `task` tool for delegating work to specialized agents
- 🧠 **Context Management**: Automatic eviction of large tool results and conversation summarization

Deep agents are inspired by applications like Claude Code, Deep Research, and Manus.

## Workshop Structure

- **Step 10**: Introduction to Deep Agents - Built-in capabilities
- **Step 11**: Subagents - Specialized delegation
- **Step 12**: Custom Middleware - Extending deep agents

---

## Setup

## Model Configuration

Choose which model to use for the agents. By default, we use `gpt-4o-mini`, but you can uncomment and use other models as needed.

In [9]:
# Model initialization - choose your model
# Default: gpt-4o-mini (recommended for most use cases)

from langchain.chat_models import init_chat_model
from langchain_openai import ChatOpenAI

# Option 1: OpenAI GPT-4o-mini (default, vision-capable)
# model = init_chat_model("gpt-4o-mini", temperature=0)

# Option 2: Anthropic Claude Sonnet 4.5 (uncomment to use)
model = init_chat_model("claude-sonnet-4-5", temperature=0)

# Option 3: GPT OSS 120B (uncomment to use)
# model = ChatOpenAI(
#     model="openai/gpt-oss-120b",
#     base_url="https://your-path-to-gpt-oss-120b.dev/v1",
#     api_key="dummy_api_key",
#     default_headers={"X-API-KEY": "dummy_api_key"}
# )

print(f"✅ Model initialized: {model.__class__.__name__}")

✅ Model initialized: ChatAnthropic


In [10]:
# Install required packages (if needed)
# !pip install deepagents langchain langchain-openai langgraph python-dotenv

In [11]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

---

# Step 10: Introduction to Deep Agents

## Key Concepts

Instead of `create_agent`, we use `create_deep_agent` which automatically includes:

1. **TodoListMiddleware** - Provides `write_todos` tool for planning
2. **FilesystemMiddleware** - Provides file system tools for context management
3. **SubAgentMiddleware** - Provides `task` tool for delegation

## Comparison

```python
# Regular agent (Steps 1-9)
agent = create_agent(
    model=model,
    tools=[read_calendar, write_calendar],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=checkpointer,
)

# Deep agent (Step 10+)
agent = create_deep_agent(
    model=model,
    tools=[read_calendar, write_calendar],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=checkpointer,
    # Automatically includes: write_todos, file system, task tool
)
```

In [12]:
from deepagents import create_deep_agent
from langchain.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from typing import List, Dict

# Mock calendar storage
_calendar_events: List[Dict] = []

@tool
def read_calendar(date: str = None) -> str:
    """Read calendar events. If date is provided, filter events for that date."""
    if date:
        filtered = [e for e in _calendar_events if e.get("date") == date]
        if not filtered:
            return f"No events found for {date}"
        return "\n".join([f"- {e['title']} on {e['date']} at {e['time']} in {e.get('location', 'N/A')}"
                         for e in filtered])
    
    if not _calendar_events:
        return "No events in calendar"
    
    return "\n".join([f"- {e['title']} on {e['date']} at {e['time']} in {e.get('location', 'N/A')}"
                     for e in _calendar_events])

@tool
def write_calendar(title: str, date: str, time: str, location: str = "") -> str:
    """Create a new calendar event."""
    for event in _calendar_events:
        if event["date"] == date and event["time"] == time:
            return f"Conflict: There's already an event '{event['title']}' scheduled for {date} at {time}"
    
    new_event = {"title": title, "date": date, "time": time, "location": location}
    _calendar_events.append(new_event)
    return f"Successfully created event '{title}' on {date} at {time} in {location}"

# Initialize checkpointer
# Uses the model initialized in the Model Configuration section above
checkpointer = MemorySaver()

# System prompt
SYSTEM_PROMPT = """You are a helpful calendar assistant with advanced planning capabilities. You can:
- Read calendar events using read_calendar
- Create new events using write_calendar

Built-in Deep Agent Capabilities:
- write_todos: Break down complex tasks into steps (automatically available)
- File system tools: ls, read_file, write_file, edit_file (automatically available)
  * Use these to store notes, drafts, or large amounts of information
  * Files are stored in agent state and persist within the conversation thread
- Task delegation: Spawn subagents for complex subtasks (automatically available)

When handling multi-step requests:
1. Use write_todos to plan your approach
2. Check the calendar first for conflicts
3. Use file system tools to save notes or drafts if needed
4. Execute the plan step by step

Be friendly and confirm when events are successfully created."""

# Create the deep agent
# Uses the model initialized in the Model Configuration section above
agent_10 = create_deep_agent(
    model=model,  # Use the model from Model Configuration section
    tools=[read_calendar, write_calendar],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=checkpointer,
)

print("✅ Deep agent created with built-in capabilities!")

✅ Deep agent created with built-in capabilities!


### Test the Deep Agent

In [13]:
# Complex multi-step request
config = {"configurable": {"thread_id": "deep-agent-thread-1"}}

result = agent_10.invoke({
    "messages": [{
        "role": "user",
        "content": "Schedule three team meetings for next week: Monday at 2 PM, Wednesday at 3 PM, and Friday at 4 PM. All in the main conference room. After that, check my calendar and return a report of what I have scheduled."
    }]
}, config=config)

print("Agent Response:")
print(result['messages'][-1].content)

# Check if todos were used
if "todos" in result:
    print("\n📋 Agent's Plan (from write_todos):")
    for todo in result["todos"]:
        status_emoji = "✅" if todo["status"] == "completed" else "⏳" if todo["status"] == "in_progress" else "📝"
        print(f"  {status_emoji} {todo['content']}")

Agent Response:
## ✅ All Done!

**Meetings Successfully Scheduled:**

1. **Monday, January 6, 2025** - Team Meeting at 2:00 PM in Main Conference Room
2. **Wednesday, January 8, 2025** - Team Meeting at 3:00 PM in Main Conference Room
3. **Friday, January 10, 2025** - Team Meeting at 4:00 PM in Main Conference Room

**Calendar Report:**
Your calendar currently shows these three team meetings scheduled for next week. All meetings are in the Main Conference Room as requested.


### Test Memory

In [14]:
# Test memory - the agent should remember what we just scheduled
result2 = agent_10.invoke({
    "messages": [{"role": "user", "content": "What meetings did we just schedule?"}]
}, config=config)

print("Agent Response:")
print(result2['messages'][-1].content)

Agent Response:
We just scheduled three team meetings for next week:

1. **Monday, January 6, 2025** at 2:00 PM in Main Conference Room
2. **Wednesday, January 8, 2025** at 3:00 PM in Main Conference Room
3. **Friday, January 10, 2025** at 4:00 PM in Main Conference Room

All three are titled "Team Meeting" and are located in the Main Conference Room.


---

# Step 11: Deep Agents with Subagents

## Key Concepts

Deep agents can delegate work to specialized **subagents** using the built-in `task` tool:

- **research-specialist**: Web search and event research
- **calendar-specialist**: Complex calendar operations
- **general-purpose**: Built-in subagent with all main agent's tools

## Benefits of Subagents

1. 🧹 **Context isolation** - Subagent work doesn't clutter main agent's context
2. 🎯 **Specialization** - Each subagent has specific tools and instructions
3. 📦 **Token efficiency** - Main agent receives only final results
4. 🔄 **Parallel potential** - Multiple subagents can work concurrently

## Subagent Configuration

```python
subagent = {
    "name": "research-specialist",
    "description": "When to use this subagent",
    "system_prompt": "How this subagent should behave",
    "tools": [web_search],
    "model": "gpt-4o-mini",
}
```

In [15]:
# Reset calendar for this example
_calendar_events.clear()

# Mock web search tool
@tool
def web_search(query: str, max_results: int = 3) -> str:
    """Search the web for events and information."""
    return f"""Search results for "{query}":

1. Classical Concert at Paris Arts Center - Dec 20, 2025 at 7 PM
   Korean Symphony Orchestra performing Beethoven's 9th Symphony
   Tickets available at artscouncil.kr

2. Jazz Night at Blue Note - Dec 21, 2025 at 9 PM
   Live jazz featuring international artists
   Reservations: bluenote-Paris.com

3. K-Pop Festival - Dec 22, 2025 at 6 PM
   Outdoor festival with multiple performers
   Free entry, Han River Park"""

# Define specialized subagents
# Uses the model initialized in the Model Configuration section above
research_subagent = {
    "name": "research-specialist",
    "description": "Conducts in-depth research on events, concerts, and activities using web search. Use when you need detailed information that requires multiple searches.",
    "system_prompt": """You are an expert researcher specialized in finding events and activities.

Your job is to:
1. Break down the research question into searchable queries
2. Use web_search to gather information
3. Save detailed findings to /research_notes.txt using write_file
4. Return a concise summary (2-3 paragraphs max)

IMPORTANT: Keep your final response under 300 words to maintain clean context.
Use the file system to store detailed research, then return only the summary.""",
    "tools": [web_search],
    "model": model,  # Use the model from Model Configuration section
}

calendar_specialist_subagent = {
    "name": "calendar-specialist",
    "description": "Handles complex calendar operations including conflict resolution and multi-event scheduling. Use for scheduling multiple events or resolving conflicts.",
    "system_prompt": """You are a calendar specialist who handles complex scheduling tasks.

Your job is to:
1. Use write_todos to plan your approach
2. Check calendar for conflicts using read_calendar
3. Schedule events one by one using write_calendar
4. Handle conflicts by suggesting alternative times
5. Save scheduling notes to /scheduling_log.txt if needed

Return a summary of what was scheduled and any conflicts encountered.""",
    "tools": [read_calendar, write_calendar],
    "model": model,  # Use the model from Model Configuration section
}

# System prompt for the main agent
SUPERVISOR_PROMPT = """You are a supervisor calendar assistant that coordinates between specialized subagents.

Available subagents (use the 'task' tool to delegate):
1. research-specialist: For finding events, concerts, and activities via web search
2. calendar-specialist: For complex calendar operations and multi-event scheduling
3. general-purpose: A general subagent with all your tools (automatically available)

When to delegate:
- Use research-specialist when you need to search for events or gather information
- Use calendar-specialist for scheduling multiple events or handling complex calendar tasks
- Use general-purpose for other multi-step tasks that would clutter your context

Workflow for "find and schedule" requests:
1. Delegate to research-specialist to find events
2. Review the research summary
3. Delegate to calendar-specialist to handle scheduling
4. Confirm with the user

This keeps your context clean while still going deep on subtasks."""

# Create the deep agent with subagents
# Uses the model initialized in the Model Configuration section above
agent_11 = create_deep_agent(
    model=model,  # Use the model from Model Configuration section
    tools=[read_calendar, write_calendar, web_search],
    system_prompt=SUPERVISOR_PROMPT,
    checkpointer=checkpointer,
    subagents=[research_subagent, calendar_specialist_subagent],
)

print("✅ Deep agent with subagents created!")
print("   - research-specialist")
print("   - calendar-specialist")
print("   - general-purpose (built-in)")

✅ Deep agent with subagents created!
   - research-specialist
   - calendar-specialist
   - general-purpose (built-in)


### Test Subagent Delegation

In [16]:
config = {"configurable": {"thread_id": "subagent-demo-1"}}

# Complex request that benefits from subagents
result = agent_11.invoke({
    "messages": [{
        "role": "user",
        "content": "Find upcoming concerts in Paris this month February 2026 and schedule the classical concert in my calendar"
    }]
}, config=config)

print("Agent Response:")
print(result['messages'][-1].content)

# Check if files were created by subagents
if "files" in result:
    print("\n📁 Files created by subagents:")
    for path in result["files"].keys():
        print(f"   {path}")

Agent Response:
I apologize, but I wasn't able to find specific concert listings for Paris in February 2026. Concert schedules for that timeframe haven't been published yet, as venues and artists typically announce events only 3-6 months in advance.

**What I found:**
- Concert schedules for February 2026 are not yet available online
- Major classical music venues in Paris include:
  - Philharmonie de Paris
  - Opéra Garnier
  - Théâtre des Champs-Élysées
  - Salle Pleyel

**Recommendations:**
1. Check back in late 2025 (around November/December) when February 2026 schedules will be published
2. Visit the websites of major classical venues directly
3. Use ticketing platforms like Fnac Spectacles or Ticketmaster France

Would you like me to:
- Help you set a reminder to check for concerts closer to the date?
- Search for concerts in a different month that might already be scheduled?
- Look for classical concerts in a different city?

📁 Files created by subagents:
   /research_notes.txt


---

# Step 12: Deep Agents with Custom Middleware

## Key Concepts

Deep agents come with built-in middleware:
- **TodoListMiddleware** - Planning with `write_todos`
- **FilesystemMiddleware** - File operations
- **SubAgentMiddleware** - Subagent delegation

You can add custom middleware on top:
- **HumanInTheLoopMiddleware** (via `interrupt_on` parameter)
- **SecurityGuardrailMiddleware** (custom)
- Any other custom middleware

## Middleware Execution Order

```
Custom Middleware (before_agent)
  ↓
Built-in TodoListMiddleware
  ↓
Built-in FilesystemMiddleware
  ↓
Built-in SubAgentMiddleware
  ↓
Tool Execution
  ↓
Built-in Middleware (after_tools)
  ↓
Custom Middleware (after_tools)
```

In [17]:
# Reset calendar
_calendar_events.clear()

@tool
def delete_calendar_event(date: str, time: str) -> str:
    """Delete a calendar event. This is a sensitive operation that requires confirmation."""
    for event in _calendar_events:
        if event["date"] == date and event["time"] == time:
            title = event["title"]
            _calendar_events.remove(event)
            return f"Successfully deleted event '{title}' on {date} at {time}"
    return f"No event found for {date} at {time}"

# System prompt
SYSTEM_PROMPT_12 = """You are a calendar assistant with planning and file system capabilities.

Available operations:
- read_calendar: Read calendar events
- write_calendar: Create new events
- delete_calendar_event: Delete events (requires confirmation)

Built-in deep agent tools:
- write_todos: Plan multi-step tasks
- File system: ls, read_file, write_file, edit_file
- task: Delegate to subagents

Use write_todos for multi-step operations and file system to save notes or drafts."""

# Create deep agent with interrupt_on for tool confirmation
# Uses the model initialized in the Model Configuration section above
agent_12 = create_deep_agent(
    model=model,  # Use the model from Model Configuration section
    tools=[read_calendar, write_calendar, delete_calendar_event],
    system_prompt=SYSTEM_PROMPT_12,
    checkpointer=checkpointer,
    # Add human-in-the-loop for sensitive operations
    interrupt_on={
        "delete_calendar_event": True,  # Require confirmation for deletes
    },
)

print("✅ Deep agent with custom middleware created!")
print("   - Built-in: TodoListMiddleware, FilesystemMiddleware, SubAgentMiddleware")
print("   - Custom: HumanInTheLoopMiddleware (via interrupt_on)")

✅ Deep agent with custom middleware created!
   - Built-in: TodoListMiddleware, FilesystemMiddleware, SubAgentMiddleware
   - Custom: HumanInTheLoopMiddleware (via interrupt_on)


### Test Normal Operation

In [18]:
# Pre-populate calendar
_calendar_events.append({
    "title": "Team Standup",
    "date": "2025-12-20",
    "time": "10:00",
    "location": "Zoom"
})

config = {"configurable": {"thread_id": "middleware-demo-1"}}

result = agent_12.invoke({
    "messages": [{"role": "user", "content": "What's on my calendar?"}]
}, config=config)

print("Agent Response:")
print(result['messages'][-1].content)

Agent Response:
You have one event on your calendar:

- **Team Standup** on December 20, 2025 at 10:00 AM (Location: Zoom)


### Test Human-in-the-Loop (Tool Confirmation)

In [19]:
from langgraph.types import Command

# Request to delete an event
result = agent_12.invoke({
    "messages": [{"role": "user", "content": "Delete the meeting on 2025-12-20 at 10:00"}]
}, config=config)

# Check for interrupt
if "__interrupt__" in result:
    print("🔔 Agent interrupted! Waiting for confirmation...")
    interrupt_info = result["__interrupt__"][0]
    print(f"   Tool: {interrupt_info.value['action_requests'][0]['name']}")
    print(f"   Args: {interrupt_info.value['action_requests'][0]['args']}")
    print("\n   Simulating user approval...")
    
    # Resume with approval
    result = agent_12.invoke(
        Command(resume={"decisions": [{"type": "approve"}]}),
        config=config
    )
    
    print("\nAgent Response after approval:")
    print(result['messages'][-1].content)
else:
    print("Agent Response:")
    print(result['messages'][-1].content)

🔔 Agent interrupted! Waiting for confirmation...
   Tool: delete_calendar_event
   Args: {'date': '2025-12-20', 'time': '10:00'}

   Simulating user approval...

Agent Response after approval:
Done! I've successfully deleted the "Team Standup" meeting on December 20, 2025 at 10:00 AM. Your calendar is now clear.


---

# Deep Agents CLI

The **Deep Agents CLI** is an interactive command-line interface for building with Deep Agents. It's an open-source coding assistant that runs in your terminal and retains persistent memory across sessions.

## Key Features

- 📁 **File operations** - Read, write, and edit files in your project
- 🖥️ **Shell command execution** - Execute shell commands to run tests, build projects, manage dependencies
- 🔍 **Web search** - Search the web for up-to-date information (requires Tavily API key)
- 🌐 **HTTP requests** - Make HTTP requests to APIs and external services
- 📋 **Task planning** - Break down complex tasks into discrete steps with built-in todo system
- 🧠 **Memory storage** - Store and retrieve information across sessions, learning project conventions
- 👤 **Human-in-the-loop** - Require approval for sensitive tool operations

## Quick Start

1. **Set your API key** (export as environment variable or in `.env` file):
   ```bash
   export ANTHROPIC_API_KEY="your-api-key"
   # Or use OPENAI_API_KEY or GOOGLE_API_KEY
   ```

2. **Run the CLI**:
   ```bash
   # Install globally
   uv tool install deepagents-cli
   deepagents
   
   # Or run without global install
   uvx deepagents-cli
   ```

3. **Give the agent a task**:
   ```
   > Create a Python script that prints "Hello, World!"
   ```
   The agent proposes changes with diffs for your approval before modifying files.

## Configuration

- **Named agents**: Use `--agent NAME` for separate memory per agent
- **Auto-approve**: Use `--auto-approve` to skip confirmation prompts (toggle with `Ctrl+T`)
- **Resume sessions**: Use `--resume` or `-r` to resume the most recent session
- **Remote sandboxes**: Use `--sandbox TYPE` to execute in isolated environments (Modal, Daytona, Runloop)

## Interactive Features

- **Slash commands**: `/tokens`, `/clear`, `/exit`, `/help`, `/threads`, `/version`
- **Bash commands**: Prefix with `!` to execute shell commands directly: `!git status`
- **Keyboard shortcuts**: `Ctrl+E` for external editor, `Shift+Tab` to toggle auto-approve, `@filename` for auto-complete

## Memory System

Agents store information in `~/.deepagents/AGENT_NAME/memories/` as markdown files. The agent:
1. **Researches** memory for relevant context before starting tasks
2. **Checks** memory when uncertain during execution
3. **Learns** by automatically saving new information for future sessions

When you teach the agent conventions:
```
> Our API uses snake_case and includes created_at/updated_at timestamps
```

It remembers for future sessions and applies conventions automatically.

## Remote Sandboxes

Execute code in isolated remote environments for safety and flexibility:
- **Safety**: Protect your local machine from potentially harmful code
- **Clean environments**: Use specific dependencies or OS configurations
- **Parallel execution**: Run multiple agents simultaneously
- **Long-running tasks**: Execute time-intensive operations without blocking

Configure and use:
```bash
# Set up sandbox provider (Runloop, Daytona, or Modal)
export RUNLOOP_API_KEY="your-key"

# Run with sandbox
uvx deepagents-cli --sandbox runloop --sandbox-setup ./setup.sh
```

For more details, see the [Deep Agents CLI documentation](https://docs.langchain.com/oss/deepagents/cli).

---

# Summary: Deep Agents vs Regular Agents

## Regular Agents (`create_agent`)
- ✅ Simple, single-purpose agents
- ✅ Fine-grained control over execution
- ✅ Lightweight applications
- ✅ Direct tool-calling patterns
- ⚠️ Manually add all middleware
- ⚠️ Manual context management
- ⚠️ No built-in planning or file system

## Deep Agents (`create_deep_agent`)
- ✅ Complex, multi-step tasks
- ✅ Built-in planning with `write_todos`
- ✅ Built-in file system for context management
- ✅ Built-in subagent delegation
- ✅ Automatic context eviction and summarization
- ⚠️ More overhead for simple tasks
- ⚠️ Less control over execution details

## When to Use Each

**Use Regular Agents:**
- Simple, focused agents
- Learning agent fundamentals
- Lightweight applications
- Direct tool calling is sufficient

**Use Deep Agents:**
- Complex, multi-step tasks
- Large context management
- Tasks requiring subagent delegation
- Long-running research or analysis
- Production agents inspired by Claude Code, Deep Research

---

## Further Reading

- **Deep Agents Documentation**: `docs/DEEP_AGENTS.md`
- **Workshop Progression**: `docs/deep_agents_progression.md`
- **LangChain Docs**: [docs.langchain.com](https://docs.langchain.com)
- **Deep Agents Package**: [pypi.org/project/deepagents/](https://pypi.org/project/deepagents/)